In [3]:
import os

import torch
import torchaudio
import numpy as np

from scipy.io import wavfile
import scipy.signal as sps
%pip install soundfile
import soundfile

from pydub import AudioSegment
%pip install speechbrain
from speechbrain.inference.VAD import VAD

# Define device for torch
use_cuda = True
print("CUDA is available:", torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

Note: you may need to restart the kernel to use updated packages.
  Using cached ruamel.yaml-0.18.10-py3-none-any.whl.metadata (23 kB)
   ---------------------------------------- 0.0/864.1 kB ? eta -:--:--
   --------------------------------------  860.2/864.1 kB 26.6 MB/s eta 0:00:01
   --------------------------------------- 864.1/864.1 kB 18.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/481.4 kB ? eta -:--:--
   --------------------------------------- 481.4/481.4 kB 15.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   --------------------------------------- 992.0/992.0 kB 31.7 MB/s eta 0:00:00
Using cached ruamel.yaml-0.18.10-py3-none-any.whl (117 kB)
   ---------------------------------------- 0.0/115.5 kB ? eta -:--:--
   ---------------------------------------- 115.5/115.5 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: ruamel.yaml
    Found existing installation: ruamel.yaml 0.17.21
    Uninstalling ruamel.yaml-0.17

SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://speechbrain.readthedocs.io/en/latest/audioloading.html


CUDA is available: False


In [ ]:
from pydub.utils import which
from pydub import AudioSegment

AudioSegment.converter = "/usr/bin/ffmpeg"
print("FFmpeg path:", AudioSegment.converter)

FFmpeg path: None


In [7]:

def reencode(file_name: str, target_name: str):
    """reencodes the given .mp3 file to .wav"""
    AudioSegment.from_mp3(file_name).export(target_name, format="wav")

def reencode_all():
    """reencodes all .mp3 files in data/mp3"""
    for lang in os.listdir("data/mp3"):
        source_dir = "data/mp3/" + lang
        target_dir = "data/wav/" + lang
        try:
            os.makedirs(target_dir)
        except FileExistsError:
            pass
        except:
            raise ValueError("file system error")

        for file in os.listdir(source_dir):
            if file.endswith(".mp3"):
                try:
                    reencode(source_dir + '/' + file, target_dir + '/' + file[:-3] + "wav")
                except Exception as e:
                    print(file + " in " + source_dir + " reencoding failed due to", e)
                    break

        print("reencoding for " + lang + " done!")
    print("reencoding done!")

def resample(file_name: str, target: int):
    """resamples the given file to the target sample rate"""
    wf, sr = torchaudio.load(file_name)
    transform = torchaudio.transforms.Resample(sr, target)
    new_wf = transform(wf)
    torchaudio.save(file_name, new_wf, target)

def resample_all(target: int):
    """resamples all .wav files in data/"""
    for lang in os.listdir("data/wav"):
        source_dir = "data/wav/" + lang

        # resample each wavfile
        for file in os.listdir(source_dir):
            if file.endswith(".wav"):
                try:
                    resample(source_dir + '/' + file, target)
                except:
                    print(file + " in " + source_dir + " resampling failed!")

        print("resampling for " + lang + " done!")
    print("resampling done!")

def generate_spectrograms(length: float):
    """generates all spectrograms from data/"""
    # load voice detection model to split audio
    vad = VAD.from_hparams(source="speechbrain/vad-crdnn-libriparty", savedir="pretrained_models/vad-crdnn-libriparty")

    for lang in os.listdir("data/wav"):
        source_dir = "data/wav/" + lang
        target_dir = "data/mel/" + lang
        try:
            os.makedirs(target_dir)
        except FileExistsError:
            pass
        except:
            raise ValueError("file system error")
        subseg_file_number = 0

        for file in os.listdir(source_dir):
            if file.endswith(".wav"):
                # load audio file and get segments
                file_path = source_dir + '/' + file
                wf, sr = torchaudio.load(file_path)
                sample_count = len(wf[0])
                duration = sample_count / sr
                maxseg_len = int(sr * length)
                segments = get_boundaries(vad, file_path)

                # split audio to valid sub-segments
                for segment in segments:
                    segment_len = segment[1] - segment[0]
                    segstart_idx = int(sample_count * (segment[0] / duration))
                    subseg_count = int(segment_len / length)

                    for i in range(subseg_count):
                        # extract segment and generate spectrogram
                        subseg = wf[:, segstart_idx + i * maxseg_len:segstart_idx + (i + 1) * maxseg_len]
                        subseg_spec = torchaudio.transforms.AmplitudeToDB()(to_spectrogram(subseg, sr)) # AmplitudeToDB gives log-mel spectrogram
                        save_spectrogram(target_dir + '/' + lang + str(subseg_file_number) + ".npy", subseg_spec)
                        subseg_file_number += 1

        print("spectrograms for " + lang + " done!")
    print("all spectrograms done!")

def get_boundaries(vad, file_name):
    """gets a tensor of pairs demarcating vocal segments from a model and file"""
    return vad.get_speech_segments(file_name)

def to_spectrogram(data, sr):
    """generates spectrogram from waveform slice"""
    spec_transform = torchaudio.transforms.MelSpectrogram(sr, n_mels=128)
    spec = spec_transform(data)
    return spec

def save_spectrogram(file_name, spec):
    """save spectrogram as npy image file"""
    np.save(file_name, spec.numpy())

def read_spectrogram(file_name):
    """reads a .npy spectrogram to a torch.Tensor"""
    return torch.tensor(np.load(file_name))

if __name__ == "__main__":
    # this script works; expecting all the mp3 audio input to be in `data/mp3/<lang>`
    # it will iterate through all <lang> to generate the log-mel specs for each
    reencode_all()
    resample_all(16000)
    generate_spectrograms(3)


c:\Users\muthu\anaconda3\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


common_voice_zh-CN_18524189.mp3 in data/mp3/cn reencoding failed due to [WinError 2] The system cannot find the file specified
reencoding for cn done!
common_voice_en_41910499.mp3 in data/mp3/en reencoding failed due to [WinError 2] The system cannot find the file specified
reencoding for en done!
reencoding done!
resampling for cn done!
resampling for en done!
resampling done!


c:\Users\muthu\anaconda3\Lib\site-packages\speechbrain\utils\fetching.py:151: UserWarning: Using SYMLINK strategy on Windows for fetching potentially requires elevated privileges and is not recommended. See `LocalStrategy` documentation.
  warnings.warn(


OSError: [WinError 1314] A required privilege is not held by the client: 'C:\\Users\\muthu\\.cache\\huggingface\\hub\\models--speechbrain--vad-crdnn-libriparty\\snapshots\\c5d5ae4fce161d94c3ab0286e32fb4a041a21a04\\hyperparams.yaml' -> 'c:\\Users\\muthu\\Downloads\\group_19_tpdl\\pretrained_models\\vad-crdnn-libriparty\\hyperparams.yaml'